# Logic

**Logic** is the art and science of effective reasoning. Logic is used to investigate how to draw general and reliable conclusions from a collection of facts.

Examples of logics include:
- propositional logic
- first-order logic
- higher-order logic
- modal logics

In propositional logic, the connectives ($\land$, $\lor$, $\neg$, $\to$) have a fixed interpretation. The constants (propositional variables) $p$, $q$, $r$ may be interpreted at will.

The precedence order is:
- $\neg$
- $\land$
- $\lor$
- $\to$

A formula is **satisfiable** it it has an interpretation that makes it true. The interpretation is the **model** of the formula. A formula is **unsatisfiable** if it does not have any model.

A formula is **valid** if it is true in any interpretation. A propositional logic formula is valid iff its negation is unsatisfiable.

- $p \lor q \to q \lor p$ is valid (i.e. true for any interpretation)
- $p \lor q \to q$ is satisfiable (i.e. true for certain interpretations, i.e. $p$ is false)
- $p \land \neg q \land (\neg p \lor q)$ is unsatisfiable (i.e. no interpretation makes it true)

Two formulas $F$ and $G$ are **equivalent** iff they evaluate to the same value (true or false) in every interpretation).

## Setup

This Python Jupyter notebook uses Z3, which is a high performance theorem prover developed at Microsoft Research. Z3 has been used in many applications, such as hardware/software verification, constraint solving and security.

Z3 is a Satisfiability Modulo Theories (SMT) solver that checks whether a given logical formula $F$ is satisfiable.

To use this notebook, run:

```
pip install z3-solver
```

In [1]:
import z3

## Z3 examples

### Calculator

What is $z = 3 + 2$?

In [2]:
x, y, z = z3.Ints("x y z")
z3.solve(x==2, y==3, z==x+y)

[z = 5, y = 3, x = 2]


If $x + 3 = 17$, what is $x$?

In [3]:
x = z3.Int("x")
z3.solve(x + 3 == 17)

[x = 14]


### Exporting the expression in SMT-LIB format

In [4]:
x, y, z = z3.Ints("x y z")
s = z3.Solver()
s.add(x==2, y==3, z==x+y)
print(s.sexpr())

(declare-fun x () Int)
(declare-fun y () Int)
(declare-fun z () Int)
(assert (= x 2))
(assert (= y 3))
(assert (= z (+ x y)))



### Constraint satisfaction using Z3

To solve the problem $x + 2 y = 7$ subject to $x > 2$ and $y < 10$ where $x, y \in \mathbb{Z}$:

In [5]:
x = z3.Int("x")  # integer variable named 'x'
y = z3.Int("y")
z3.solve(x>2, y<10, x+2*y==7)  # solve a system of constraints

[y = 0, x = 7]


**Dog, cat, mouse problem**: A dog costs \\$15, a cat \\$1 and a mouse \\$0.25. Spend \\$100 to buy 100 animals, where each animal must be bought at least once.

In [6]:
dog, cat, mouse = z3.Ints('dog cat mouse')
z3.solve(
    dog >= 1,                               # at least one dog
    cat >= 1,                               # at least one cat
    mouse >= 1,                             # at least one mouse
    dog + cat + mouse == 100,               # 100 animals
    1500*dog + 100*cat + 25*mouse == 10000  # $100 = 10,000c spent
)

[mouse = 56, cat = 41, dog = 3]


**XKCD 287** problem: How many of each appetizer should be chosen to bring the total to \$15.05?

![XKCD 287](https://imgs.xkcd.com/comics/np_complete.png)

In [7]:
a,b,c,d,e,f = z3.Ints("a b c d e f")
s = z3.Solver()
s.add(a >= 0)
s.add(b >= 0)
s.add(c >= 0)
s.add(d >= 0)
s.add(e >= 0)
s.add(f >= 0)
s.add(a*215 + b*275 + c*335 + d*355 + e*420 + f*580 == 1505)

while s.check() == z3.sat:
    
    # Display the solution
    m = s.model()
    print(m)
    
    # Block the current solution and start again
    s.add(z3.Not(z3.And(a == m[a], 
                        b == m[b],
                        c == m[c],
                        d == m[d],
                        e == m[e],
                        f == m[f] )))

[d = 0, b = 0, a = 7, f = 0, c = 0, e = 0]
[c = 0, d = 2, e = 0, f = 1, b = 0, a = 1]


**Kinematics problem**: Suppose a vehicle is traveling at 30m/s and applies the brakes, decelerating at 8m/s, how far does it travel to a stop?

In [8]:
d, a, t, v_i, v_f = z3.Reals('d a t v__i v__f')

equations = [
   d == v_i * t + (a*t**2)/2,
   v_f == v_i + a*t,
]

problem = [
    v_i == 30,
    v_f == 0,
    a == -8
]

z3.solve(equations + problem)

[a = -8, v__f = 0, v__i = 30, t = 15/4, d = 225/4]


Constraints can be added to the solver using the `add` method (the constraints are said to be **asserted** in the solver). The method `check` solves the asserted constraints. The result is `sat` (satisfiable) if a solution was found, otherwise `unsat` (unsatisfiable) if no solution exists.

In [9]:
s = z3.Solver()
s.add(equations)  # add constraints
s.add(problem)
s.check()         # find a solution
s.model()[d]

225/4

In [10]:
s.statistics()

(:eliminated-vars 5
 :max-memory      21.47
 :memory          18.96
 :num-allocs      42682459
 :rlimit-count    29452
 :time            0.01)

### Formula / expression simplification

Simplify $x + y + 2x + 3$:

In [11]:
x = z3.Int("x")
y = z3.Int("y")
z3.simplify(x + y + 2*x + 3)

3 + 3*x + y

Simplify $x < y + x + 2$:

In [12]:
z3.simplify(x < y + x + 2)

Not(y <= -2)

$$
x < y + x + 2 \\
\implies 0 < y + 2 \\
\implies y > -2 \\\implies \neg (y \leq 2)
$$

In [13]:
z3.simplify(z3.And(x + 1 >= 3, x**2 + x**2 + y**2 + 2 >= 5))

And(x >= 2, 2*x**2 + y**2 >= 3)

### Non-linear polynomial constraints

Solve 

$$
x^2 + y^2 > 3 \\
x^3 + y < 5 \\
\text{subject to } x,y \in \mathbb{R}
$$

In [14]:
x = z3.Real('x')  # create a real variable
y = z3.Real('y')
z3.solve(x**2 + y**2 > 3, x**3 + y < 5)

[y = 2, x = 1/8]


### Rational numbers

In [15]:
z3.Q(1,3)

1/3

In [16]:
x + z3.Q(1,3)

x + 1/3

### Unsatisfiable system of constraints

If the system of constraints if unsatisfiable, `no solution` is returned by the solver.

In [17]:
x = z3.Real('x')
z3.solve(x > 4, x < 0)

no solution


### Boolean logic

In [18]:
p = z3.Bool('p')
q = z3.Bool('q')

In [19]:
z3.Not(p)

Not(p)

In [20]:
z3.And(p,q)

And(p, q)

In [21]:
z3.solve(z3.And(p,q))

[p = True, q = True]


In [22]:
z3.Or(p,q)

Or(p, q)

In [23]:
z3.solve(z3.Or(p,q))

[p = True, q = False]


Note how Z3 only finds one solution.

In [24]:
r = z3.Bool('r')
z3.solve(z3.Implies(p, q), r == z3.Not(q), z3.Or(z3.Not(p), r))

[p = False, q = True, r = False]


Simplify $p \land q \land \text{True}$:

In [25]:
z3.simplify(z3.And(p, q, True))

And(p, q)

Simplify $p \lor q \lor \text{True}$:

In [26]:
z3.simplify(z3.Or(p, q, True))

True

Solve $p \lor q \to q$

In [27]:
p, q = z3.Bools('p q')
z3.solve(z3.Implies(z3.Or(p,q), q))

[p = False, q = False]


Is $p \lor q \to q$ valid?

In [28]:
z3.solve(z3.Not(z3.Implies(z3.Or(p,q), q)))

[p = True, q = False]


As its negation is satisfiable, it is not valid. A propositional logic formula is valid iff its negation is unsatisfiable.

Is $p \land \neg q \land (\neg p \lor q)$ satisfiable?

In [29]:
p, q = z3.Bools('p q')
z3.And(p, z3.Not(q), z3.Or(z3.Not(p), q))

And(p, Not(q), Or(Not(p), q))

In [30]:
z3.solve(z3.And(p, z3.Not(q), z3.Or(z3.Not(p), q)))

no solution


As there is no solution, then $p \land \neg q \land (\neg p \lor q)$ is unsatisfiable.

### Polynomial and Boolean constraints

Solve

$$
x < 5 \lor x > 10 \\
p \lor x^2 == 2 \\
\neg p
$$

In [31]:
p = z3.Bool('p')
x = z3.Real('x')
z3.solve(z3.Or(x < 5, x > 10), z3.Or(p, x**2 == 2), z3.Not(p))

[x = -1.4142135623?, p = False]


### Functions

Functions in Z3 have no side effects and are **total** (i.e. defined over all input values).

In [32]:
x = z3.Int('x')
y = z3.Int('y')
f = z3.Function('f', z3.IntSort(), z3.IntSort())

Function $f$ applied twice to $x$ results in $x$, but $f$ applied once is different to $x$: 

In [33]:
z3.solve(f(f(x)) == x, f(x) == y, x != y)

[x = 0, y = 1, f = [1 -> 0, else -> 1]]


The above means $f(0) = 1$ and $f(1) = 0$ and $f(a) = 1$ where $a \neq 0 \land a \neq 1$.

### Satisfiability and validity

* **Validity** 
    - finding a proof of a statement
    - a formula $F$ is a valid if $F$ always evaluates to true for any assignment of uniterpreted symbols
    - consider a formula $F$ containing $a$ and $b$. $F$ is deemed valid if it is always true for any combination of values of $a$ and $b$. 
    
* **Satisfiable** 
    - finding a solution to a set of constraints
    - a formula $F$ is satisfiable if there is some assignment of values of uninterpreted symbols under which $F$ evaluates to true

In [34]:
p, q = z3.Bools('p q')  # define two Booleans in one step
demorgan = z3.And(p, q) == z3.Not(z3.Or(z3.Not(p), z3.Not(q)))
demorgan

And(p, q) == Not(Or(Not(p), Not(q)))

In [35]:
def prove(f):
    s = z3.Solver()
    s.add(z3.Not(f))  # note the NOT
    if s.check() == z3.unsat:
        print("proved")
    else:
        print("failed to prove")

In [36]:
prove(demorgan)

proved


### Expressions, sorts and declarations

Every expression has a **sort** (type).

In [37]:
x = z3.Int("x")
print(x.sort())
print((x+1).sort())
print((x >= 1).sort())

Int
Int
Bool


### Arrays

Arrays appear to have more similiarity to Python dicts than to lists.

In [38]:
# Array from integer to integer
A = z3.Array('A', z3.IntSort(), z3.IntSort())
x = z3.Int('x')
A[x]

A[x]

In [39]:
z3.Select(A, x)  # same as A[x]

A[x]

In [40]:
z3.Store(A, x, 10)

Store(A, x, 10)

In [41]:
A[x]

A[x]

### Quantifiers

Z3 can solve quantifier-free problems containing arithmetic, bit-vectors, Booleans, arrays, functions and datatypes.

In [42]:
f = z3.Function('f', z3.IntSort(), z3.IntSort(), z3.IntSort())
x, y = z3.Ints('x y')
z3.ForAll([x, y], f(x,y) == 0)

ForAll([x, y], f(x, y) == 0)

In [43]:
z3.Exists(x, f(x, x) >= 0)

Exists(x, f(x, x) >= 0)

In [44]:
a, b = z3.Ints('a b')
z3.solve(z3.ForAll(x, f(x,x) == 0), f(a,b) == 1)

[b = 2, a = 0, f = [(0, 2) -> 1, else -> 0]]


**Model an object-orientated system with single inheritance**:

The predicate for sub-typing is defined as $\text{subtype}(x,y)$, which means that $x$ is a sub-type of $y$.

Type $x$ is a sub-type of itself:

$$
\forall x: \text{subtype}(x,x)
$$

If $x$ is a sub-type of $y$ and $y$ is a sub-type of $z$ then $x$ is a sub-type of $z$:

$$
\forall x,y: \text{subtype}(x,y) \land \text{subtype}(y,z) \to \text{subtype}(x,z)
$$

If $x$ is a sub-type of $y$ and $y$ is a sub-type of $x$ then $x = y$:

$$
\forall x,y: \text{subtype}(x,y) \land \text{subtype}(y,x) \to x = y
$$

If $x$ is a sub-type of $y$ and $x$ is a sub-type of $z$ then either $y$ is a sub-type of $z$ or $z$ is a sub-type of $y$:

$$
\forall x,y,z: \text{subtype}(x,y) \land \text{subtype}(x,z) \to \text{subtype}(y,z) \lor \text{subtype}(z,y)
$$

All types $x$ are a sub-type of a root type:

$$
\forall x: \text{subtype}(x, \text{root})
$$

In [46]:
Type = z3.DeclareSort('Type')
subtype = z3.Function('subtype', Type, Type, z3.BoolSort())
root = z3.Const('root', Type)
x, y, z = z3.Consts('x y z', Type)

In [47]:
x.sort()

Type

In [54]:
axioms = [
    z3.ForAll(x, subtype(x, x)),
    z3.ForAll([x,y,z], z3.Implies(
        z3.And(subtype(x,y), subtype(y,z)),  # subtype(𝑥,𝑦) ∧ subtype(𝑦,𝑧)
        subtype(x, z)
    )),
    z3.ForAll([x,y], z3.Implies(
        z3.And(subtype(x,y), subtype(y,z)),  # subtype(𝑥,𝑦) ∧ subtype(𝑦,𝑥)
        x == y
    )),
    z3.ForAll([x,y,z], z3.Implies(
        z3.And(subtype(x,y), subtype(x,z)),  # subtype(𝑥,𝑦) ∧ subtype(𝑥,𝑧)
        z3.Or(subtype(y,z), subtype(z,y))    # subtype(𝑦,𝑧) ∨ subtype(𝑧,𝑦)
    )),
    z3.ForAll(x, subtype(root, x))
]

s = z3.Solver()
s.add(axioms)
s.check()

sat

In [55]:
# Define a specific case
animal, cat = z3.Consts('animal cat', Type)
domain = [
    subtype(cat, animal)
]

s = z3.Solver()
s.add(axioms + domain)
s.check()

sat

In [57]:
animal, cat, dog = z3.Consts('animal cat dog', Type)
domain = [
    subtype(cat, animal),
    subtype(dog, animl)
]

s = z3.Solver()
s.add(axioms + domain)
s.check()

sat

## Propositional logic

**Truth tables** are useful for showing the effect of operators and for determining equivalences. All of the possible combinations of input variables are listed in the table. If there $n$ variables, there will be $2^n$ rows, thus the table grows exponentially in the number of variables.

### Logical NOT

The logical NOT operation is represented as $\neg$ and its truth table is shown below. The operation can only be performed on one variable.

| $p$  | $\neg p$   |
|------|------------|
| $0$  |     $1$    |
| $1$  |     $0$    |



### Logical AND

The logical AND operator is represented as $\land$. Two or more variables are required and the result will only be true if both variables are true.

| $p$   | $q$   | $p \land q$   |
|-------|-------|---------------|
|  $0$  |  $0$  |      $0$      |
|  $0$  |  $1$  |      $0$      |
|  $1$  |  $0$  |      $0$      |
|  $1$  |  $1$  |      $1$      |

### Logical OR

The logical OR operator is represented as $\lor$. Two or more variables are required and the operator returns true if one of more variables are true.

| $p$   | $q$   | $p \lor q$    |
|-------|-------|---------------|
|  $0$  |  $0$  |      $0$      |
|  $0$  |  $1$  |      $1$      |
|  $1$  |  $0$  |      $1$      |
|  $1$  |  $1$  |      $1$      |

### Logical implication

Logical implication is represented as $\to$.

| $p$   | $q$   | $p \to q$     |
|-------|-------|---------------|
|  $0$  |  $0$  |      $1$      |
|  $0$  |  $1$  |      $1$      |
|  $1$  |  $0$  |      $0$      |
|  $1$  |  $1$  |      $1$      |

Consider $p \to q$. The operator returns true if the initial condition $p$ is false, regardless of the value of $q$.

$p \to q$ is equivalent to $\neg p \lor q$, as can be shown by the truth table:

| $p$   | $q$   | $\neg p$ | $\neg p \lor q$     | $p \to q$     |
|-------|-------|----------|---------------------|---------------|
|  $0$  |  $0$  | $1$      |      $1$            |      $1$      |
|  $0$  |  $1$  | $1$      |      $1$            |      $1$      |
|  $1$  |  $0$  | $0$      |      $0$            |      $0$      |
|  $1$  |  $1$  | $0$      |      $1$            |      $1$      |

### Logical equivalence

Logical equivalence is represented as $\leftrightarrow$. The operator returns true if both variables have the same value.

| $p$   | $q$   | $p \leftrightarrow q$     |
|-------|-------|---------------------------|
|  $0$  |  $0$  |      $1$                  |
|  $0$  |  $1$  |      $0$                  |
|  $1$  |  $0$  |      $0$                  |
|  $1$  |  $1$  |      $1$                  |

### Tautology

A tautology is when all values are true.

$p \leftrightarrow \neg \neg p$ is a tautology.

| $p$   | $\neg \neg p$   | $p \leftrightarrow \neg \neg p$     |
|-------|-------|---------------------------|
|  $0$  |  $0$  |      $1$                  |
|  $0$  |  $0$  |      $1$                  |
|  $1$  |  $1$  |      $1$                  |
|  $1$  |  $1$  |      $1$                  |

## First order logic

### Male or female example

In [143]:
NameSort = z3.DeclareSort('NameSort')
male = z3.Function('male', NameSort, z3.BoolSort())
female = z3.Function('female', NameSort, z3.BoolSort())
chris = z3.Const('chris', NameSort)

x, y, z = z3.Consts('x y z', NameSort)

s = z3.Solver()
s.add(z3.ForAll(x, z3.Xor(male(x), female(x))))
s.add(male(chris))
s.check()

sat

In [144]:
s.add(female(chris))
print(s.check())
#s.model()

unsat


### Backward chaining

A backward chaining algorithm starts from the goal and uses the rules to find facts that support the goal.

## Bibliography

z3:

https://ericpony.github.io/z3py-tutorial/guide-examples.htm

https://sat-smt.codes/SAT_SMT_by_example.pdf

https://ericpony.github.io/z3py-tutorial/advanced-examples.htm